In [1]:
import numpy as np
import pandas as pd
# Scanning data
#import os
#for dirname, _, filenames in os.walk('./kaggle'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [2]:
# Open raw datasets
orders = pd.read_csv("./kaggle/kaggle_tab_1345/tab_1_orders.csv")
categories = pd.read_csv("./kaggle/kaggle_tab_1345/tab_3_categories.csv")
users = pd.read_csv("./kaggle/kaggle_tab_1345/tab_4_user_profiles.csv")
products_props = pd.read_csv("./kaggle/kaggle_tab_1345/tab_5_product_properties.csv")
cities = pd.read_csv("./kaggle/tab_6_city.csv")

products = pd.read_csv("./kaggle/products_full.csv")

In [ ]:
#items: product_id | title | description (set of properties)
#train: user_id | items (product_id, rating)
#benchmark: user_id | last_purchased_product_id 

In [3]:
# Formatting dates
orders['order_created_time'] = pd.to_datetime(orders['order_created_time'], format='%Y-%m-%d %H:%M:%S')
# Merging cities into orders
orders_c = pd.merge(orders, cities, on='store_id')

orders_c.head()

,user_id,order_id,order_created_time,retailer,store_id,platform,city_name
0,72,17431000,2020-09-26 10:48:57,METRO,21,app,Москва
1,1474,10066159,2020-05-14 17:35:58,METRO,21,app,Москва
2,1474,10083198,2020-05-15 04:40:50,METRO,21,app,Москва
3,1474,15090928,2020-08-16 09:32:13,METRO,21,app,Москва
4,2657,7652238,2020-03-29 19:44:24,METRO,21,app,Москва


In [4]:
### FUNCTIONS

# Return list of cities with orders
def orders_by_city():
    cities_array = cities.city_name.unique()
    df_dict = {}
    for city in cities_array:
        df_dict[city] = orders_c[orders_c.city_name == city]
    return df_dict, cities_array

# Return bill to the order
def bill_builder(order_id):
    return products[products.order_id == order_id]

# Return user info
def user_info(user_id):
    info = {}
    info["qty"] = orders[orders.user_id == user_id].order_id.count()
    info["city"] = orders_c[orders_c.user_id == user_id].city_name.mode()[0]
    return info

# Product properties
def get_product_props(product_id):
    return products_props[products_props.product_id == product_id]

def get_user_orders(user_id):
    return orders[orders.user_id == user_id]

### FUNCTIONS

In [6]:
city_orders, cities_list = orders_by_city()

In [7]:
cities_list

array(['Москва', 'Санкт-Петербург', 'Тюмень', 'Нижний Новгород',
       'Московская Область', 'Ростов-на-Дону', 'Краснодар', 'Самара',
       'Казань', 'Омск', 'Волгоград', 'Екатеринбург', 'Уфа', 'Воронеж',
       'Новосибирск', 'Рязань', 'Пермь', 'Тольятти', 'Кемерово',
       'Ставрополь', 'Ярославль', 'Челябинск', 'Калининград', 'Ульяновск',
       'Стерлитамак', 'Ижевск', 'Липецк', 'Кострома', 'Набережные Челны',
       'Чебоксары', 'Тула', 'Новороссийск', 'Вологда', 'Пенза', 'Сургут',
       'Смоленск', ' Каменск-Уральский', 'Калуга', 'Владикавказ',
       'Красноярск', 'Новокузнецк', 'Белгород', 'Владимир', 'Орел',
       'Саранск', 'Оренбург', 'Иркутск', 'Астрахань', 'Иваново',
       'Йошкар-Ола', 'Черкесск', 'Тверь', 'Курск', 'Барнаул', 'Тобольск',
       'Саратов', 'Магнитогорск', 'Майкоп', 'Альметьевск', 'Мурманск',
       'Курган', 'Архангельск', 'Ноябрьск', 'Томск', 'Ханты-Мансийск',
       'Тамбов', 'Брянск', 'Петрозаводск', 'Чита', 'Нижний Тагил',
       'Абакан', 'Сыкты

In [8]:
moscow = city_orders[cities_list[0]]

In [9]:
# Moscow analyzing
orders_per_customers = moscow.groupby(['user_id'])['order_id'].count()

In [10]:
orders_per_customers[orders_per_customers > 20] 

user_id
766        25
1578       27
1724       25
2268       30
2865       21
           ..
2528488    26
2578283    21
2628660    21
2831548    23
2987260    26
Name: order_id, Length: 2408, dtype: int64

In [11]:
moscow.head()

,user_id,order_id,order_created_time,retailer,store_id,platform,city_name
0,72,17431000,2020-09-26 10:48:57,METRO,21,app,Москва
1,1474,10066159,2020-05-14 17:35:58,METRO,21,app,Москва
2,1474,10083198,2020-05-15 04:40:50,METRO,21,app,Москва
3,1474,15090928,2020-08-16 09:32:13,METRO,21,app,Москва
4,2657,7652238,2020-03-29 19:44:24,METRO,21,app,Москва


In [ ]:
products["master_category_id"] = products["master_category_id"].fillna(0).astype(int)
products[products.user_id == 1474]

In [12]:
### Product features
products.head()

,user_id,order_id,price,quantity,discount,product_id,master_category_id,parent_category_id,cost_real
0,525,5354800,116.000000,4,0.00,37548,101.0,99,116.000000
1,525,5354800,56.990002,2,13.08,5636,596.0,51,43.910002
2,525,5354800,64.989998,2,40.02,22107,596.0,51,24.969997
3,525,5354800,44.990002,6,0.00,2530,77.0,74,44.990002
4,525,5354800,65.900002,6,30.99,3818486,76.0,74,34.910002


In [5]:
# Prepare products dataset
print(len(products))
### Remove NaN value
products = products[products.product_id != 0]
### Improve discounts
products.loc[products['discount'] < 0, 'discount'] = 0
### Add the features
products['cost_real'] = (products.price - products.discount)
products['wholesaler'] = (products.quantity >= 20)
products['technics'] = (products.cost_real > 5000) & (products.quantity < 5) 
### Set master_category_id to int
products["master_category_id"] = products["master_category_id"].fillna(0).astype(int)

len(products)

59596980


58984303

In [6]:
# Get all users products
users_to_products = products.groupby(['user_id']).agg({
    'product_id': lambda x: list(set(x))
})

In [8]:
users_to_products.to_csv("user_x_product.csv")

In [90]:
products.head()

,user_id,order_id,price,quantity,discount,product_id,master_category_id,parent_category_id,cost_real,wholesaler,technics
0,525,5354800,116.000000,4,0.00,37548,101,99,116.000000,False,False
1,525,5354800,56.990002,2,13.08,5636,596,51,43.910002,False,False
2,525,5354800,64.989998,2,40.02,22107,596,51,24.969997,False,False
3,525,5354800,44.990002,6,0.00,2530,77,74,44.990002,False,False
4,525,5354800,65.900002,6,30.99,3818486,76,74,34.910002,False,False


In [96]:
unique = products.groupby(['product_id']).agg({
    'cost_real':'mean', 
    'technics': 'first', 
    'master_category_id':'first', 
    'parent_category_id': 'first',
    'order_id': 'count'}
)

In [97]:
unique.tail()

,cost_real,technics,master_category_id,parent_category_id,order_id
product_id,,,,,
13550929,177.909091,False,574,9,11
13550935,222.181389,False,91,90,187
13550945,346.185602,False,85,84,50
13550974,319.000000,False,36,35,8
13551016,720.207273,False,647,112,11


In [98]:
unique.to_csv('unique_products.csv')
del[unique]

In [20]:
del[products]

In [101]:
### User features
user_order = orders_c[['user_id', 'order_id', 'retailer', 'city_name']]
user_features = user_order.groupby('user_id').agg(
        orders=pd.NamedAgg(column='order_id', aggfunc=lambda x: x.count()),
        retailer=pd.NamedAgg(column='retailer', aggfunc=lambda x: x.mode().iat[0]),
        city_name=pd.NamedAgg(column='city_name', aggfunc=lambda x: x.mode().iat[0]),
)

In [102]:
user_features.head()

,orders,retailer,city_name
user_id,,,
72,1,METRO,Москва
83,1,METRO,Московская Область
142,1,METRO,Московская Область
187,1,ВкусВилл,Москва
224,1,Ашан,Москва


In [103]:
user_features.to_csv('user_features_basic.csv')

In [105]:
# Calculate total discount per order
print("Calculating discounts")
discounts_sum = products.groupby(['user_id'],as_index = False)['discount'].sum()
discounts1= pd.merge(products,discounts_sum,on = 'user_id')
discounts1 = discounts1.rename(columns = {'discount_x':'discount', 'discount_y' :'discount_total'})
products = discounts1
# Does the user prefer discounts?
products['prefer_discounts'] = (products.discount_total > products.discount_total.mean())
print("Discounts calculated!")
products['total'] = products['cost_real']*products['quantity']

Calculating discounts
Discounts calculated!
Starting aggregation...


AttributeError: 'SeriesGroupBy' object has no attribute 'tolist'

In [107]:
print("Starting aggregation...")
user_features2 = products.groupby(['user_id']).agg({
    'total':'mean', 
    'prefer_discounts': 'max', 
    'master_category_id':lambda x: list(set(x)), 
    'parent_category_id':lambda x: list(set(x))}
)
print("Finished")

Starting aggregation...
Finished


In [108]:
user_features2

,total,prefer_discounts,master_category_id,parent_category_id
user_id,,,,
51,343.234517,True,"[113, 596, 91, 30, 85, 24, 33, 33, 21, 85, 596...","[112, 51, 90, 20, 84, 20, 20, 20, 20, 84, 51, ..."
65,207.623441,False,"[30, 30, 91, 60, 591, 30, 91, 91, 91, 257, 97,...","[20, 20, 90, 56, 90, 20, 90, 90, 90, 252, 94, ..."
83,170.171765,False,"[432, 432, 432, 432, 434, 435, 328, 343, 343, ...","[429, 429, 429, 429, 429, 429, 321, 290, 290, ..."
142,3344.000000,False,[581],[9]
187,277.110001,False,"[85, 91]","[84, 90]"
...,...,...,...,...
3142901,147.935000,False,"[217, 625, 625, 626]","[216, 67, 67, 67]"
3142924,123.057505,False,"[30, 366, 366, 122, 370, 384, 360, 379]","[20, 365, 365, 119, 365, 378, 358, 377]"
3142936,89.434446,False,"[644, 644, 643, 610, 37, 132, 33, 33, 33]","[99, 99, 99, 19, 35, 479, 20, 20, 20]"


In [109]:
user_features

,orders,retailer,city_name
user_id,,,
72,1,METRO,Москва
83,1,METRO,Московская Область
142,1,METRO,Московская Область
187,1,ВкусВилл,Москва
224,1,Ашан,Москва
...,...,...,...
3432957,1,Лента,Иркутск
3432966,1,Лента,Новосибирск
3432977,1,Лента,Томск


In [111]:
z = pd.merge(user_features, user_features2, on='user_id')

In [112]:
z.to_csv('user_features_full.csv')

In [110]:
del[products, products_props]

In [113]:
z.head()

,orders,retailer,city_name,total,prefer_discounts,master_category_id,parent_category_id
user_id,,,,,,,
83,1,METRO,Московская Область,170.171765,False,"[432, 432, 432, 432, 434, 435, 328, 343, 343, ...","[429, 429, 429, 429, 429, 429, 321, 290, 290, ..."
142,1,METRO,Московская Область,3344.000000,False,[581],[9]
187,1,ВкусВилл,Москва,277.110001,False,"[85, 91]","[84, 90]"
224,1,Ашан,Москва,636.272493,False,"[61, 61, 61, 598, 600, 600, 630, 40, 91, 91, 9...","[56, 56, 56, 51, 56, 56, 74, 35, 90, 90, 90, 90]"
260,2,METRO,Москва,220.464167,False,"[76, 76, 76, 76, 630, 630, 630, 630, 80, 110, ...","[74, 74, 74, 74, 74, 74, 74, 74, 74, 99, 99, 9..."


In [9]:
test = pd.read_csv("./unique_products.csv")
test = test.sort_values(by=['order_id'], ascending=False)
test.head()

,product_id,cost_real,technics,master_category_id,parent_category_id,order_id
194,709,83.309392,False,91,90,368956
9329,39590,47.022403,False,29,20,112981
71,166,39.501508,False,26,20,98763
10661,55133,124.119817,False,85,84,96574
32972,3497419,79.130221,False,91,90,89520


In [15]:
sum20 = (test.head(round(len(test)*0.2)).cost_real*test.head(round(len(test)*0.2)).order_id).sum()
print(sum20)

5198952382.693323


In [16]:
sum_all = (test.cost_real*test.order_id).sum()
print(sum_all)

6481204217.838449


In [17]:
print(sum20/sum_all*100, "%")

80.2158396488119 %


In [4]:
# Shrinking dataset
paths = [
    "./kaggle/sbermarket_tab_2_1/tab_2_products_2020-01-01.csv",
    "./kaggle/sbermarket_tab_2_1/tab_2_products_2020-01-11.csv",
    "./kaggle/sbermarket_tab_2_1/tab_2_products_2020-01-21.csv",
    "./kaggle/sbermarket_tab_2_1/tab_2_products_2020-01-31.csv",
    "./kaggle/sbermarket_tab_2_1/tab_2_products_2020-02-10.csv",
    "./kaggle/sbermarket_tab_2_1/tab_2_products_2020-02-20.csv",
    "./kaggle/sbermarket_tab_2_1/tab_2_products_2020-03-01.csv",
    "./kaggle/sbermarket_tab_2_1/tab_2_products_2020-03-11.csv",
    "./kaggle/sbermarket_tab_2_1/tab_2_products_2020-03-21.csv",
    "./kaggle/sbermarket_tab_2_1/tab_2_products_2020-03-31.csv",
    "./kaggle/sbermarket_tab_2_2/tab_2_products_2020-04-10.csv",
    "./kaggle/sbermarket_tab_2_2/tab_2_products_2020-04-20.csv",
    "./kaggle/sbermarket_tab_2_2/tab_2_products_2020-04-30.csv",
    "./kaggle/sbermarket_tab_2_3/tab_2_products_2020-05-10.csv",
    "./kaggle/sbermarket_tab_2_3/tab_2_products_2020-05-20.csv",
    "./kaggle/sbermarket_tab_2_3/tab_2_products_2020-05-30.csv",
    "./kaggle/sbermarket_tab_2_4/tab_2_products_2020-06-09.csv",
    "./kaggle/sbermarket_tab_2_4/tab_2_products_2020-06-19.csv",
    "./kaggle/sbermarket_tab_2_4/tab_2_products_2020-06-29.csv",
    "./kaggle/sbermarket_tab_2_5/tab_2_products_2020-07-09.csv",
    "./kaggle/sbermarket_tab_2_5/tab_2_products_2020-07-19.csv",
    "./kaggle/sbermarket_tab_2_5/tab_2_products_2020-07-29.csv",
    "./kaggle/sbermarket_tab_2_6/tab_2_products_2020-08-08.csv",
    "./kaggle/sbermarket_tab_2_6/tab_2_products_2020-08-18.csv",
    "./kaggle/sbermarket_tab_2_6/tab_2_products_2020-08-28.csv",
    "./kaggle/sbermarket_tab_2_6/tab_2_products_2020-09-07.csv"
]
df = pd.read_csv('./products.csv')
i = 1
for file in paths:
    print("File #", i, file)
    temp = pd.read_csv(file)
    temp = temp[['user_id', 'order_id', 'price', 'quantity', 'discount', 'product_id', 'master_category_id', 'parent_category_id']]
    #df.to_csv('./products.csv', mode='a', header=False)
    with open('./products_full.csv', 'a') as f:
        temp.to_csv(f, header=False)
        print("File appended!")
    i+=1;
    del[temp]

File # 1 ./kaggle/sbermarket_tab_2_1/tab_2_products_2020-01-01.csv
File appended!
File # 2 ./kaggle/sbermarket_tab_2_1/tab_2_products_2020-01-11.csv
File appended!
File # 3 ./kaggle/sbermarket_tab_2_1/tab_2_products_2020-01-21.csv
File appended!
File # 4 ./kaggle/sbermarket_tab_2_1/tab_2_products_2020-01-31.csv
File appended!
File # 5 ./kaggle/sbermarket_tab_2_1/tab_2_products_2020-02-10.csv
File appended!
File # 6 ./kaggle/sbermarket_tab_2_1/tab_2_products_2020-02-20.csv
File appended!
File # 7 ./kaggle/sbermarket_tab_2_1/tab_2_products_2020-03-01.csv
File appended!
File # 8 ./kaggle/sbermarket_tab_2_1/tab_2_products_2020-03-11.csv
File appended!
File # 9 ./kaggle/sbermarket_tab_2_1/tab_2_products_2020-03-21.csv
File appended!
File # 10 ./kaggle/sbermarket_tab_2_1/tab_2_products_2020-03-31.csv
File appended!
File # 11 ./kaggle/sbermarket_tab_2_2/tab_2_products_2020-04-10.csv
File appended!
File # 12 ./kaggle/sbermarket_tab_2_2/tab_2_products_2020-04-20.csv
File appended!
File # 13 ./k

In [87]:
df = pd.DataFrame(columns=['user_id', 
                           'order_id', 
                           'price', 
                           'quantity', 
                           'discount', 
                           'product_id', 
                           'master_category_id', 
                           'parent_category_id'])
df.to_csv('products.csv', index=False)

In [100]:
import os
print(os.getcwd())

C:\Users\nmiha\Documents\Projects\hack-lab-5-rec-system


In [6]:
for df in pd.read_csv('products_full.csv',sep=',', chunksize=10):
    print(df.head(10))
    break

   user_id  order_id       price  quantity    discount  product_id  \
0      525   5354800  116.000000         4    0.000000       37548   
1      525   5354800   56.990002         2   13.080000        5636   
2      525   5354800   64.989998         2   40.020000       22107   
3      525   5354800   44.990002         6    0.000000        2530   
4      525   5354800   65.900002         6   30.990000     3818486   
5      525   5354800   29.900000         1   39.990002         687   
6      525   5354800  239.000000         1   91.000000       64913   
7      525   5354800  448.459991         1  125.580002        4372   
8      525   5354800  410.470001         1  188.220001       16673   
9      525   5354800   38.990002         6    0.000000       67252   

   master_category_id  parent_category_id  
0               101.0                  99  
1               596.0                  51  
2               596.0                  51  
3                77.0                  74  
4        

In [5]:
# Delete all DataFrames
del[orders, cities, products, products_props, categories, orders_c, city_orders, moscow]

NameError: name 'city_orders' is not defined

In [ ]:
# End